In [1]:
import pandas as pd
import numpy as np

from pandas.core.dtypes.common import is_numeric_dtype, is_string_dtype

In [2]:
def get_numeric_col(df):
    numeric_col_list = []

    for col_name in df.columns:
        if is_numeric_dtype(df[col_name].dtypes):
            numeric_col_list.append(col_name)

    return numeric_col_list


def get_string_col(df):
    string_col_list = []

    for col_name in df.columns:
        if is_string_dtype(df[col_name].dtypes):
            string_col_list.append(col_name)

    return string_col_list


def outlier_iqr(df):
    numeric_col_list = get_numeric_col(df)
    total_size = df.shape[0]
    total_outlier_size = 0
    for col_name in numeric_col_list:
        q1, q3 = np.percentile(df[col_name], [25, 75])

        iqr = q3 - q1

        lower_bound = q1 - (iqr * 1.5)
        upper_bound = q3 + (iqr * 1.5)

        count = 0
        count += df[upper_bound < df[col_name]].shape[0]
        count += df[df[col_name] < lower_bound].shape[0]

        if count < total_size * 0.025:
            df = df[upper_bound >= df[col_name]]
            df = df[df[col_name] >= lower_bound]
            total_outlier_size += count

    return total_outlier_size, df

In [3]:
def check_dataset(df):
    print(df.info())
    print()
    print(f'number of "NULL" value: {df.isnull().sum().sum()}')
    df_drop_NAN = df.dropna(axis=0)
    print(f'droped row : {df.shape[0] -df_drop_NAN.shape[0]}', end='\n\n')

    num_outlier, df_drop_outlier = outlier_iqr(df_drop_NAN.copy())
    print(f'number of outlier : {num_outlier} / {df_drop_NAN.shape[0]} ---> {df_drop_NAN.shape[0] - num_outlier}')

    return df_drop_outlier

In [4]:
def makePriceToClass(df):
    df.loc[df['price']<=10000,'class']='A'
    df.loc[(df['price']>10000) & (df['price']<=20000),'class']='B'
    df.loc[(df['price']>20000) & (df['price']<=30000),'class']='C'
    df.loc[(df['price']>30000) & (df['price']<=50000),'class']='D'
    df.loc[(df['price']>50000) ,'class']='E'
    df.drop('price',axis=1)
    return df

In [5]:
df = pd.read_csv(r'.\dataset\vehicles.csv')

In [6]:
print(df.shape[0])
null_count_series = df.isnull().sum()
print(null_count_series)

426880
id                   0
url                  0
region               0
region_url           0
price                0
year              1205
manufacturer     17646
model             5277
condition       174104
cylinders       177678
fuel              3013
odometer          4400
title_status      8242
transmission      2556
VIN             161042
drive           130567
size            306361
type             92858
paint_color     130203
image_url           68
description         70
county          426880
state                0
lat               6549
long              6549
posting_date        68
dtype: int64


In [20]:
year_threshold = 1990
print(df[df['year'] < year_threshold].shape[0])
df_after_thresh = df[df['year'] >= year_threshold]
print(df_after_thresh.head(10))

12383
            id                                                url  region  \
27  7316814884  https://auburn.craigslist.org/ctd/d/auburn-uni...  auburn   
28  7316814758  https://auburn.craigslist.org/ctd/d/auburn-uni...  auburn   
29  7316814989  https://auburn.craigslist.org/ctd/d/auburn-uni...  auburn   
30  7316743432  https://auburn.craigslist.org/ctd/d/auburn-uni...  auburn   
31  7316356412  https://auburn.craigslist.org/cto/d/auburn-uni...  auburn   
32  7316343444  https://auburn.craigslist.org/ctd/d/auburn-uni...  auburn   
33  7316304717  https://auburn.craigslist.org/ctd/d/auburn-uni...  auburn   
34  7316285779  https://auburn.craigslist.org/cto/d/auburn-201...  auburn   
35  7316257769  https://auburn.craigslist.org/ctd/d/auburn-uni...  auburn   
36  7316133914  https://auburn.craigslist.org/ctd/d/auburn-uni...  auburn   

                       region_url  price    year manufacturer  \
27  https://auburn.craigslist.org  33590  2014.0          gmc   
28  https://aubu

In [9]:
print(df_after_thresh.shape[0])
null_count_series = df_after_thresh.isnull().sum()
print(null_count_series)

413292
id                   0
url                  0
region               0
region_url           0
price                0
year                 0
manufacturer     14229
model             4146
condition       169290
cylinders       173639
fuel              2729
odometer          4293
title_status      7914
transmission      2465
VIN             150311
drive           125603
size            298001
type             87471
paint_color     125342
image_url            0
description          2
county          413292
state                0
lat               6451
long              6451
posting_date         0
dtype: int64


In [23]:
# ['condition', 'cylinders', 'VIN', 'drive', 'size', 'paint_color', 'county']
has_many_null_col_list = list(null_count_series[null_count_series > df.shape[0]/2].index)
print(has_many_null_col_list)
drop_col_list = ['url', 'image_url', 'posting_date', 'description', 'region_url', 'id', 'lat', 'long']
drop_col_list += has_many_null_col_list

df_drop_unusful = df_after_thresh.drop(drop_col_list, axis=1)

df_drop_outlier = check_dataset(df_drop_unusful.copy())

['size', 'county']
<class 'pandas.core.frame.DataFrame'>
Int64Index: 413292 entries, 27 to 426879
Data columns (total 16 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   region        413292 non-null  object 
 1   price         413292 non-null  int64  
 2   year          413292 non-null  float64
 3   manufacturer  399063 non-null  object 
 4   model         409146 non-null  object 
 5   condition     244002 non-null  object 
 6   cylinders     239653 non-null  object 
 7   fuel          410563 non-null  object 
 8   odometer      408999 non-null  float64
 9   title_status  405378 non-null  object 
 10  transmission  410827 non-null  object 
 11  VIN           262981 non-null  object 
 12  drive         287689 non-null  object 
 13  type          325821 non-null  object 
 14  paint_color   287950 non-null  object 
 15  state         413292 non-null  object 
dtypes: float64(2), int64(1), object(13)
memory usage: 53.6+ MB
None

number 

In [24]:
for col_name in get_string_col(df_drop_outlier):
        if df_drop_outlier[col_name].dtype == 'object':
            print(f'{col_name}(categorical column) has "{len(df_drop_outlier[col_name].unique())}" different value')

region(categorical column) has "398" different value
manufacturer(categorical column) has "39" different value
model(categorical column) has "6038" different value
condition(categorical column) has "6" different value
cylinders(categorical column) has "8" different value
fuel(categorical column) has "5" different value
title_status(categorical column) has "6" different value
transmission(categorical column) has "3" different value
VIN(categorical column) has "30029" different value
drive(categorical column) has "3" different value
type(categorical column) has "13" different value
paint_color(categorical column) has "12" different value
state(categorical column) has "51" different value


In [25]:
# print(df_after_thresh['model'].unique())
df_drop_outlier.drop(['model', 'VIN'], axis=1, inplace=True)

In [29]:
df_drop_outlier.reset_index(drop=True, inplace=True)
df_drop_outlier.to_csv(r'./dataset/afterPreprocessing.csv')

In [28]:
print(df_drop_outlier.head(10))
print(df_drop_outlier.shape[0])

   region  price    year manufacturer condition    cylinders   fuel  odometer  \
0  auburn  27990  2012.0          gmc      good  8 cylinders    gas   68696.0   
1  auburn  34590  2016.0    chevrolet      good  6 cylinders    gas   29499.0   
2  auburn  29990  2016.0    chevrolet      good  6 cylinders    gas   17302.0   
3  auburn  38590  2011.0    chevrolet      good  8 cylinders    gas   30237.0   
4  auburn  32990  2017.0         jeep      good  6 cylinders    gas   30041.0   
5  auburn  37990  2016.0    chevrolet      good  8 cylinders    gas    9704.0   
6  auburn  27990  2018.0       nissan      good  6 cylinders    gas   37332.0   
7  auburn  34990  2018.0         ford      good  8 cylinders    gas   18650.0   
8  auburn  29990  2014.0    chevrolet      good  8 cylinders    gas   26129.0   
9  auburn  33990  2017.0         jeep      good  6 cylinders  other   34152.0   

  title_status transmission drive    type paint_color state  
0        clean        other   4wd  pickup     

In [34]:
df_drop_outlier = pd.read_csv(r'./dataset/afterPreprocessing.csv', index_col='Unnamed: 0')
print(df_drop_outlier.head(10))

   region  price    year manufacturer fuel  odometer title_status  \
0  auburn  33590  2014.0          gmc  gas   57923.0        clean   
1  auburn  22590  2010.0    chevrolet  gas   71229.0        clean   
2  auburn  39590  2020.0    chevrolet  gas   19160.0        clean   
3  auburn  30990  2017.0       toyota  gas   41124.0        clean   
4  auburn  15000  2013.0         ford  gas  128000.0        clean   
5  auburn  27990  2012.0          gmc  gas   68696.0        clean   
6  auburn  34590  2016.0    chevrolet  gas   29499.0        clean   
7  auburn  35000  2019.0       toyota  gas   43000.0        clean   
8  auburn  29990  2016.0    chevrolet  gas   17302.0        clean   
9  auburn  38590  2011.0    chevrolet  gas   30237.0        clean   

  transmission    type state      lat       long  
0        other  pickup    al  32.5900 -85.480000  
1        other  pickup    al  32.5900 -85.480000  
2        other  pickup    al  32.5900 -85.480000  
3        other  pickup    al  32.590

In [40]:
from sklearn.preprocessing import OrdinalEncoder

encoder=OrdinalEncoder()
df_needEncoding=df_after_thresh[get_string_col(df_after_thresh)]
df_notEncoding=df_after_thresh.drop(get_string_col(df_after_thresh), axis=1)

df_encoding=pd.DataFrame(encoder.fit_transform(df_needEncoding), columns=get_string_col(df_after_thresh))

# print(df_encoding.tail(10))
# print(df_notEncoding.tail(10))

df_encoding = pd.concat([df_notEncoding, df_encoding], axis=1)

df_encoding.to_csv(r'./dataset/123.csv')

In [18]:
print(df_drop_outlier.isnull().sum())
print(df_drop_outlier.shape[0])

region          0
price           0
year            0
manufacturer    0
fuel            0
odometer        0
title_status    0
transmission    0
type            0
state           0
lat             0
long            0
dtype: int64
284392
